In [1]:
include("./src/methods.jl")
using PyPlot, JuMP

In [2]:
using MathOptInterface

In [3]:
c = [0,0,1]
A = [-1. 1. -1.;
     -1. -1. -1.;
     1. 0. 0.
     -1. 0. 0.]
b = [0,0,4.,-1.]

c_simplex, A_simplex, b_simplex,var_relation = simplex_format(c,A,b)

atol = 1e-8
max_iter = 1000
xzero = []
num_const, num_var = size(A)
isempty(xzero) ?  x = zeros(num_var) : x = xzero
if ~all(A*x - b .<= atol)
    phaseI = true
    c_phaseI,A_phaseI,b_phaseI,
        x_phaseI,bfs,nfs = phaseI_simplex_problem(A_simplex,b_simplex)
    x_simplex = SimplexFromBFS(c_phaseI,A_phaseI,
        b_phaseI,x_phaseI,index_bfs=bfs,index_nfs=nfs)
end
var_relation

Dict{Any,Any} with 3 entries:
  2 => [2, 3]
  3 => [4, 5]
  1 => [1]

In [7]:
c = [0,0,1]
A = [-1. 1. -1.;
     -1. -1. -1.;
     1. 0. 0.
     -1. 0. 0.]
b = [0,0,4.,-1.]

4-element Array{Float64,1}:
  0.0
  0.0
  4.0
 -1.0

In [8]:
model = Model(Clp.Optimizer)
@variable(model, x[1:size(c)[1]])
@constraint(model,A*x.<=b) 
@objective(model, Min, c'*x)
optimize!(model)
xo = zeros(size(c)[1])
for i = 1:size(c)[1]
    xo[i] = value(x[i])
end
xo

Coin0506I Presolve 0 (-4) rows, 0 (-3) columns and 0 (-8) elements
Clp3002W Empty problem - 0 rows, 0 columns and 0 elements
Clp0000I Optimal - objective value -4
Coin0511I After Postsolve, objective -4, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -4 - 0 iterations time 0.002, Presolve 0.00


3-element Array{Float64,1}:
  4.0
 -0.0
 -4.0

In [9]:
c = [0,0,1]
A = [-1. 1. -1.;
     -1. -1. -1.;
     1. 0. 0.
     -1. 0. 0.]
b = [0,0,4.,0.]
cs,As,bs,var_relation = simplex_format(c,A,b)
x = [0.,0,0,0,0,0,0,4]
x_simplex = SimplexFromBFS(cs,As,bs,x,index_bfs=[6,7,8],index_nfs=[1,2,3,4,5])
x_circus = variable_simplex_to_circus(var_relation,x_simplex)
x_simplex = variable_circus_to_simplex(var_relation,x_circus,cs,As,bs)
x_circus

3-element Array{Float64,1}:
  4.0
  0.0
 -4.0

#### Phase 1


In [10]:
c = [0,0,1]
A = [-1. 1. -1.;
     -1. -1. -1.;
     1. 0. 0.
     -1. 0. 0.]
b = [0,0,4.,-1.]

# x = [0,0,0,0,0,0,0,4,1]
cs,As,bs,var_relation = simplex_format(c,A,b)
cpi, Api, bpi, xpi,bfs_pi,nfs_pi = phaseI_simplex_problem(As,bs)

([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0], [-1.0 1.0 … 0.0 0.0; -1.0 -1.0 … 0.0 0.0; 1.0 0.0 … 1.0 0.0; 1.0 -0.0 … 0.0 1.0], [0.0; 0.0; 4.0; 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 1.0], [10, 11, 12, 13], [1, 2, 3, 4, 5, 6, 7, 8, 9])

In [11]:
x_simplex = SimplexFromBFS(cpi,Api,bpi,xpi,index_bfs=bfs_pi,index_nfs=nfs_pi)
x_circus = variable_simplex_to_circus(var_relation,x_simplex)

3-element Array{Float64,1}:
  1.0
  0.0
 -1.0

In [12]:
model = Model(Clp.Optimizer)
@variable(model, x[1:size(cpi)[1]])
@constraint(model,Api*x.==bpi) 
@constraint(model,x.>=zeros(size(cpi)[1])) 
@objective(model, Min, cpi'*x)
optimize!(model)
xo = zeros(size(cpi)[1])
for i = 1:size(cpi)[1]
    xo[i] = value(x[i])
end


Coin0506I Presolve 0 (-17) rows, 0 (-13) columns and 0 (-33) elements
Clp3002W Empty problem - 0 rows, 0 columns and 0 elements
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0 - 0 iterations time 0.002, Presolve 0.00


In [13]:
c = [0,0,1]
A = [-1. 1. -1.;
     -1. -1. -1.;
     1. 0. 0.
     -1. 0. 0.]
b = [0,0,4.,-1.]
# xo = x_circus

pygui(true)
px= [0,4 ,4 ,0 ,0,0,0 ,0,4,4,4 ,4]
py= [0,0 ,-4,-4,0,4,-4,4,4,-4,0,4]
pz= [0,-4,0 ,4 ,0,4,4 ,4,0,0,-4,0]
plt= PyPlot.plot3D(px,py,pz)
PyPlot.scatter3D([xo[1]],[xo[2]],[xo[3]])

PyObject <mpl_toolkits.mplot3d.art3d.Path3DCollection object at 0x7f9f3f270410>

### Checking if BFS is Optimal

In [14]:
check_optimality_simplex(cpi,Api,bpi,xo)

1

In [15]:
c = [0,0,1]
A = [-1. 1. -1.;
     -1. -1. -1.;
     1. 0. 0.
     -1. 0. 0.]
b = [0,0,4.,0.]
cs,As,bs,var_relation = simplex_format(c,A,b)
x = [0.,0,0,0,0,0,0,4]
x_simplex = SimplexFromBFS(cs,As,bs,x,index_bfs=[6,7,8],index_nfs=[1,2,3,4,5])
x_circus  = variable_simplex_to_circus(var_relation,x_simplex)
active_ix = [1,2,3]
active_ix_simplex = []
for i in active_ix
    push!(active_ix_simplex,var_relation[i][1])
end
nactive_ix_simplex = filter(x -> !(x in active_ix_simplex),collect(1:size(x_simplex)[1]))

check_optimality_simplex(cs,As,bs,x_simplex,
    index_bfs=active_ix_simplex,index_nfs=nactive_ix_simplex)

1

In [16]:
checkKKT(c,A,b,x_circus)
checkKKT(c,A,b,[0.0,0.0,0.0])

0

### Importing MPS file test case 

In [17]:
model = JuMP.read_from_file("../TestCases/afiro.mps")
set_optimizer(model,Clp.Optimizer)
optimize!(model)
variables = JuMP.all_variables(model)
X_sol = zeros(size(variables)[1])
for (i,v) in enumerate(variables)
    X_sol[i] = value(v)
end

@show objective_value(model);
@show X_sol;


list_of_constraint_types(model)

gc = JuMP.all_constraints(model,VariableRef, MOI.GreaterThan{Float64})
ec = JuMP.all_constraints(model,GenericAffExpr{Float64,VariableRef}, MOI.EqualTo{Float64})
lc = JuMP.all_constraints(model,GenericAffExpr{Float64,VariableRef}, MOI.LessThan{Float64})
cj = JuMP.objective_function(model)

variables = JuMP.all_variables(model)
A = zeros(1,size(variables)[1])
b= zeros(1)

c = zeros(1,size(variables)[1])
c[1,indexin(cj.terms.keys,variables)] = cj.terms.vals
c = c'[:]

for i in 1:size(lc)[1]
    Aux = zeros(1,size(variables)[1])
    lc_terms = constraint_object(lc[i]).func.terms
    lc_value = constraint_object(lc[i]).set.upper
    if i == 1
        A[1,indexin(lc_terms.keys,variables)] = lc_terms.vals
        b = lc_value
    else
        Aux[1,indexin(lc_terms.keys,variables)] = lc_terms.vals
        A = vcat(A,Aux)
        b = vcat(b,lc_value)
    end
end

for i in 1:size(gc)[1]
    Aux = zeros(1,size(variables)[1])
    lc_terms = constraint_object(gc[i]).func
    lc_value = constraint_object(gc[i]).set.lower
    Aux[1,findfirst(variables .== lc_terms)] = -1
    A = vcat(A,Aux)
    b = vcat(b,lc_value)
end

for i in 1:size(ec)[1]
    Aux = zeros(1,size(variables)[1])
    lc_terms = constraint_object(ec[i]).func.terms
    lc_value = constraint_object(ec[i]).set.value
    Aux[1,indexin(lc_terms.keys,variables)] = lc_terms.vals
    A = vcat(A,Aux)
    b = vcat(b,lc_value)
    
    Aux[1,indexin(lc_terms.keys,variables)] = -lc_terms.vals
    A = vcat(A,Aux)
    b = vcat(b,-lc_value)
end


objective_value(model) = -464.75314285714296
X_sol = [339.94285714285724, 0.0, 0.0, 0.0, 0.0, 54.50000000000001, 0.0, 0.0, 19.307142857142853, 0.0, 25.499999999999993, 0.0, 500.00000000000006, 18.214285714285708, 84.80000000000001, 0.0, 0.0, 24.079999999999984, 80.0, 18.214285714285708, -2.3606070955485056e-14, 215.00000000000003, 0.0, -5.684341886080802e-14, 0.0, 0.0, 0.0, 475.92000000000013, 0.0, 0.0, 0.0, 383.94285714285724]
Coin0506I Presolve 6 (-21) rows, 9 (-23) columns and 26 (-57) elements
Clp0006I 0  Obj 0 Dual inf 1,8228541 (3)
Clp0006I 3  Obj -464,75314
Clp0000I Optimal - objective value -464,75314
Coin0511I After Postsolve, objective -464,75314, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -464,7531429 - 3 iterations time 0,002, Presolve 0,00


In [18]:
model = Model(Clp.Optimizer)
@variable(model, x[1:size(c)[1]])
@constraint(model,A*x.<=b) 
@objective(model, Min, c'*x)
optimize!(model)
xo = zeros(size(c)[1])
for i = 1:size(c)[1]
    xo[i] = value(x[i])
end
xo
@show objective_value(model);

objective_value(model) = -464.75314285714285
Coin0506I Presolve 11 (-56) rows, 16 (-16) columns and 58 (-91) elements
Clp0006I 0  Obj 0 Dual inf 2,199997 (3)
Clp0006I 4  Obj -464,75314
Clp0000I Optimal - objective value -464,75314
Coin0511I After Postsolve, objective -464,75314, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -464,7531429 - 4 iterations time 0,002, Presolve 0,00


In [19]:
cs,As,bs,var_relation = simplex_format(c,A,b)
model = Model(Clp.Optimizer)
@variable(model, x[1:size(cs)[1]])
@constraint(model,As*x.==bs) 
@constraint(model,x.>=zeros(size(cs)[1])) 
@objective(model, Min, cs'*x)
optimize!(model)
@show objective_value(model);

objective_value(model) = -464.75314285714285
Coin0506I Presolve 13 (-89) rows, 20 (-47) columns and 86 (-133) elements
Clp0006I 0  Obj 0 Primal inf 688 (4) Dual inf 4,217396 (4)
Clp0006I 8  Obj -464,75314
Clp0000I Optimal - objective value -464,75314
Coin0511I After Postsolve, objective -464,75314, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -464,7531429 - 8 iterations time 0,002, Presolve 0,00


In [20]:
cpi, Api, bpi, xpi,bfs_pi,nfs_pi = phaseI_simplex_problem(As,bs)

model = Model(Clp.Optimizer)
@variable(model, x[1:size(cpi)[1]])
@constraint(model,Api*x.==bpi) 
@constraint(model,x.>=zeros(size(cpi)[1])) 
@objective(model, Min, cpi'*x)
optimize!(model)
xo = zeros(size(cpi)[1])
for i = 1:size(cpi)[1]
    xo[i] = value(x[i])
end
x_circus = variable_simplex_to_circus(var_relation,xo)
all(A*x_circus.<=b.+10^-8)

# active_ix_simplex = findall((xo.-10^-8).<=0)
# active_ix_simplex = findall((xo.≈0))
# active_ix_simplex = active_ix_simplex[1:35]
# nactive_ix_simplex = filter(x -> !(x in active_ix_simplex),collect(1:size(xo)[1]))
# check_optimality(cpi,Api,bpi,xo,index_bfs=active_ix_simplex,index_nfs=nactive_ix_simplex)


Coin0506I Presolve 13 (-124) rows, 20 (-82) columns and 86 (-203) elements
Clp0006I 0  Obj 0 Primal inf 688 (4)
Clp0006I 7  Obj 8,6676808e-16
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0 - 7 iterations time 0,002, Presolve 0,00


true

### The CIRCUS Algorithm

In [21]:
function findDirection(x, A, b, Anormed,cnormed, index_active)
    # Find the Opposite to Circumcenter direction
    # Finds indexes J such that b-Ax < ϵ
    J = index_active
    lenJ = length(J)
    # If x is interior, takes direction -c
    if isempty(J)
        return -cnormed
    end
    if lenJ == length(x)
        X = Anormed[J,:]'
        xcirc = FindCircumcenter(X)
    else
        X = Matrix([cnormed (Anormed[J,:])'])
        xcirc = FindCircumcenter(X)
    end
    return -xcirc
end

function circus_treating_vertex(c,A,b,Anormed,cnormed,x,index_active)
    if checkKKT(c,A,b,x) == 1
        return x
    end
    d = findDirection(x, A, b, Anormed,cnormed, index_active)
    α = ratiotest(x,A,b,d)
    x = x + α*d/2
    d = -c
    α = ratiotest(x,A,b,d)
    x = x + α*d
    return x
end

function purification(x, A, b, c, num_var,index_active,atol=1e-8)
    num_active = length(index_active)
    iter = 0
    xold = x
    if iszero(num_active)
        alpha = ratiotest(x,A,b,-c)
        x = x - alpha*c
        index_active = findall(b - A*x.<= atol)
        num_active = length(index_active)
    end
    
    # It is possible that the purification activates more than
    # one restriction at the same iteration, hence, there is no
    # guarantee that we can get a point in an edge
    while num_active < num_var && iter <= num_var
        iter += 1
        
        # In the Cholesky decompostion, we need to use the Hermitian function,
        # otherwise numerical errors will say the matrix is not symmetric and the
        # cholesky() function fails. Using Hermitian, we avoid this problem.
        aFact = cholesky(Hermitian(A[index_active,:]*A[index_active,:]'))
        lambda = aFact\(A[index_active,:]*c)
        d = -c +  A[index_active,:]'*lambda
        # if norm(d) ≈ 0
        #     break
        # end
        alpha = ratiotest(x,A,b,d)
        xnew = x + alpha*d
        if norm(xnew - x) < atol
            return xnew,x
        end
        xold = copy(x)
        x = xnew
        index_active = findall(b - A*x .<  atol)
        num_active = length(index_active)
    end
    return x,xold
end


function circus_acc(c, A, b, x; atol = 1e-8, max_iter = 1000)
    num_const, num_var = size(A)
    # Test to verify that xzero is a feasible starting point

    cnormed = c/norm(c)
    Anormed = copy(A)
    bnormed = copy(b)
    for  i in 1:num_const
        normA = norm(Anormed[i,:])
        Anormed[i,:] /= normA
        bnormed[i] /= normA
    end

    sortedJ = sortperm((Anormed*cnormed),rev=true)
    index_active = findall(b - A*x .<  atol)

    # If the initial point is a vertex, take the circumcenter of the
    # restrictions. Create a line until you reach a restriction.
    # From the middle of the line, move in direction -c, until you reach a restriction.
    iter = 0
    while iter <= max_iter
        iter_2 = 0
        while length(index_active) == num_var && iter_2 < 10
            if checkKKT(c,A,b,x) == 1
                return x
            end
            x = circus_treating_vertex(c,A,b,Anormed,cnormed,x,index_active)
            index_active = findall(b - A*x .<  atol)
            iter_2 += 1
        end


        (x1,x2) = purification(x,A,b,c,length(x),index_active)
        if checkKKT(c,A,b,x1) == 1
            return x1
        else
            x = x2
            index_active = findall(b - A*x .<  atol)
#             if length(index_active) < num_var
#                 x = x1
#             else
                d = findDirection(x, A, b, Anormed,cnormed, index_active)
                α = ratiotest(x,A,b,d)
                x = x + α*d
#             end
        end
        index_active = findall(b - A*x .<  atol)
        iter+=1
        @show c'*x
    end
    println("Max iterations!")
    return x
end



circus_acc (generic function with 1 method)

In [22]:
c = [0,0,1]
A = [-1. 1. -1.;
     -1. -1. -1.;
     1. 0. 0.
     0. 0. 1.
     -1. 0. 0.]
b = [0,0,4.,4.,0.0]

# xzero = [0,0,0]
xzero = [0,4,4]

x = circus_acc(c, A, b, xzero)

3-element Array{Float64,1}:
  4.0
 -7.394367382667503e-16
 -3.9999999999999996

In [23]:
# xo = [x x+d*α]
xo = x

pygui(true)
px= [0,4 ,4 ,0 ,0,0,0 ,0,4,4,4 ,4]
py= [0,0 ,-4,-4,0,4,-4,4,4,-4,0,4]
pz= [0,-4,0 ,4 ,0,4,4 ,4,0,0,-4,0]
plt= PyPlot.plot3D(px,py,pz)
PyPlot.scatter3D([xo[1,:]],[xo[2,:]],[xo[3,:]])

PyObject <mpl_toolkits.mplot3d.art3d.Path3DCollection object at 0x7f9f30e74c50>

In [25]:
model = JuMP.read_from_file("../TestCases/afiro.mps")
set_optimizer(model,Clp.Optimizer)
optimize!(model)
variables = JuMP.all_variables(model)
X_sol = zeros(size(variables)[1])
for (i,v) in enumerate(variables)
    X_sol[i] = value(v)
end

@show objective_value(model);
@show X_sol;


list_of_constraint_types(model)

gc = JuMP.all_constraints(model,VariableRef, MOI.GreaterThan{Float64})
ec = JuMP.all_constraints(model,GenericAffExpr{Float64,VariableRef}, MOI.EqualTo{Float64})
lc = JuMP.all_constraints(model,GenericAffExpr{Float64,VariableRef}, MOI.LessThan{Float64})
cj = JuMP.objective_function(model)

variables = JuMP.all_variables(model)
A = zeros(1,size(variables)[1])
b= zeros(1)

c = zeros(1,size(variables)[1])
c[1,indexin(cj.terms.keys,variables)] = cj.terms.vals
c = c'[:]

for i in 1:size(lc)[1]
    Aux = zeros(1,size(variables)[1])
    lc_terms = constraint_object(lc[i]).func.terms
    lc_value = constraint_object(lc[i]).set.upper
    if i == 1
        A[1,indexin(lc_terms.keys,variables)] = lc_terms.vals
        b = lc_value
    else
        Aux[1,indexin(lc_terms.keys,variables)] = lc_terms.vals
        A = vcat(A,Aux)
        b = vcat(b,lc_value)
    end
end

for i in 1:size(gc)[1]
    Aux = zeros(1,size(variables)[1])
    lc_terms = constraint_object(gc[i]).func
    lc_value = constraint_object(gc[i]).set.lower
    Aux[1,findfirst(variables .== lc_terms)] = -1
    A = vcat(A,Aux)
    b = vcat(b,lc_value)
end

for i in 1:size(ec)[1]
    Aux = zeros(1,size(variables)[1])
    lc_terms = constraint_object(ec[i]).func.terms
    lc_value = constraint_object(ec[i]).set.value
    Aux[1,indexin(lc_terms.keys,variables)] = lc_terms.vals
    A = vcat(A,Aux)
    b = vcat(b,lc_value)
    
    Aux[1,indexin(lc_terms.keys,variables)] = -lc_terms.vals
    A = vcat(A,Aux)
    b = vcat(b,-lc_value)
end
A = A + randn(size(A)[1],size(A)[2])*0.01;

model = Model(Clp.Optimizer)
@variable(model, x[1:size(c)[1]])
@constraint(model,A*x.<=b) 
@objective(model, Min, c'*x)
optimize!(model)
xo = zeros(size(c)[1])
for i = 1:size(c)[1]
    xo[i] = value(x[i])
end
xo
@show objective_value(model);

cs,As,bs,var_relation = simplex_format(c,A,b)
model = Model(Clp.Optimizer)
@variable(model, x[1:size(cs)[1]])
@constraint(model,As*x.==bs) 
@constraint(model,x.>=zeros(size(cs)[1])) 
@objective(model, Min, cs'*x)
optimize!(model)
@show objective_value(model);

cpi, Api, bpi, xpi,bfs_pi,nfs_pi = phaseI_simplex_problem(As,bs)

model = Model(Clp.Optimizer)
@variable(model, x[1:size(cpi)[1]])
@constraint(model,Api*x.==bpi) 
@constraint(model,x.>=zeros(size(cpi)[1])) 
@objective(model, Min, cpi'*x)
optimize!(model)
xo = zeros(size(cpi)[1])
for i = 1:size(cpi)[1]
    xo[i] = value(x[i])
end
x_circus = variable_simplex_to_circus(var_relation,xo)
all(A*x_circus.<=b.+10^-8)

objective_value(model) = -464.75314285714296
X_sol = [339.94285714285724, 0.0, 0.0, 0.0, 0.0, 54.50000000000001, 0.0, 0.0, 19.307142857142853, 0.0, 25.499999999999993, 0.0, 500.00000000000006, 18.214285714285708, 84.80000000000001, 0.0, 0.0, 24.079999999999984, 80.0, 18.214285714285708, -2.3606070955485056e-14, 215.00000000000003, 0.0, -5.684341886080802e-14, 0.0, 0.0, 0.0, 475.92000000000013, 0.0, 0.0, 0.0, 383.94285714285724]
objective_value(model) = -10.481632805367534
objective_value(model) = -10.48163258629617
Coin0506I Presolve 6 (-21) rows, 9 (-23) columns and 26 (-57) elements
Clp0006I 0  Obj 0 Dual inf 1,8228541 (3)
Clp0006I 3  Obj -464,75314
Clp0000I Optimal - objective value -464,75314
Coin0511I After Postsolve, objective -464,75314, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -464,7531429 - 3 iterations time 0,002, Presolve 0,00
Coin0506I Presolve 67 (0) rows, 32 (0) columns and 2144 (0) elements
Clp0006I 0  Obj 0 Primal inf 1363,1431 (1) Dual inf 

true

In [26]:
model = Model(Clp.Optimizer)
@variable(model, x[1:size(c)[1]])
@constraint(model,A*x.<=b) 
@objective(model, Min, c'*x)
optimize!(model)
xo = zeros(size(c)[1])
for i = 1:size(c)[1]
    xo[i] = value(x[i])
end
xo
@show objective_value(model);

objective_value(model) = -10.481632805367534
Coin0506I Presolve 67 (0) rows, 32 (0) columns and 2144 (0) elements
Clp0006I 0  Obj 0 Primal inf 1363,1431 (1) Dual inf 0,10948338 (5) w.o. free dual inf (0)
Clp0006I 31  Obj -419,73849 Primal inf 140236,84 (15)
Clp0006I 69  Obj 50,363415 Primal inf 40069,227 (14)
Clp0006I 72  Obj 2604,6199 Primal inf 83285,573 (14)
Clp0006I 72  Obj 2604,6199 Primal inf 83285,573 (14)
Clp0006I 72  Obj -6,0432429e+13 Primal inf 3,0185059e+15 (21)
Clp0006I 72  Obj -3,0216214e+14 Primal inf 1,509253e+16 (21) Dual inf 1,055857e+17 (32) w.o. free dual inf (30)
Clp0006I 88  Obj -3,0558622 Dual inf 0,21600889 (14)
Clp0006I 95  Obj -10,481633
Clp0000I Optimal - objective value -10,481633
Clp0032I Optimal objective -10,48163281 - 95 iterations time 0,002


In [40]:
function findDirection(x, A, b, Anormed,cnormed, index_active)
    # Find the Opposite to Circumcenter direction
    # Finds indexes J such that b-Ax < ϵ
    J = index_active
    lenJ = length(J)
    # If x is interior, takes direction -c
    if isempty(J)
        return -cnormed
    end
    if lenJ == length(x)
        X = Anormed[J,:]'
        xcirc = FindCircumcenter(X)
    else
        X = Matrix([cnormed (Anormed[J,:])'])
        xcirc = FindCircumcenter(X)
    end
    return -xcirc
end

function circus_treating_vertex(c,A,b,Anormed,cnormed,x,index_active)
    if checkKKT(c,A,b,x) == 1
        return x
    end
    d = findDirection(x, A, b, Anormed,cnormed, index_active)
    α = ratiotest(x,A,b,d)
    x = x + α*d/2
    d = -c
    α = ratiotest(x,A,b,d)
    x = x + α*d
    return x
end

function purification(x, A, b, c, num_var,index_active,atol=1e-8)
    num_active = length(index_active)
    iter = 0
    xold = x
    if iszero(num_active)
        alpha = ratiotest(x,A,b,-c)
        x = x - alpha*c
        index_active = findall(b - A*x.<= atol)
        num_active = length(index_active)
    end
    
    # It is possible that the purification activates more than
    # one restriction at the same iteration, hence, there is no
    # guarantee that we can get a point in an edge
    while num_active < num_var && iter <= num_var
        iter += 1
        
        # In the Cholesky decompostion, we need to use the Hermitian function,
        # otherwise numerical errors will say the matrix is not symmetric and the
        # cholesky() function fails. Using Hermitian, we avoid this problem.
        aFact = cholesky(Hermitian(A[index_active,:]*A[index_active,:]'))
        lambda = aFact\(A[index_active,:]*c)
        d = -c +  A[index_active,:]'*lambda
        # if norm(d) ≈ 0
        #     break
        # end
        alpha = ratiotest(x,A,b,d)
        xnew = x + alpha*d
#         if norm(xnew - x) < atol
#             return xnew,x
#         end
        xold = copy(x)
        x = xnew
        index_active = findall(b - A*x .<  atol)
        @show num_active = length(index_active)
    end
    println("fim")
    return x,xold
end


function circus_acc(c, A, b, x; atol = 1e-8, max_iter = 1000)
    num_const, num_var = size(A)
    # Test to verify that xzero is a feasible starting point

    cnormed = c/norm(c)
    Anormed = copy(A)
    bnormed = copy(b)
    for  i in 1:num_const
        normA = norm(Anormed[i,:])
        Anormed[i,:] /= normA
        bnormed[i] /= normA
    end

    sortedJ = sortperm((Anormed*cnormed),rev=true)
    index_active = findall(b - A*x .<  atol)

    # If the initial point is a vertex, take the circumcenter of the
    # restrictions. Create a line until you reach a restriction.
    # From the middle of the line, move in direction -c, until you reach a restriction.
    iter = 0
    while iter <= max_iter
        iter_2 = 0
        while length(index_active) == num_var && iter_2 < 10
            if checkKKT(c,A,b,x) == 1
                return x
            end
            x = circus_treating_vertex(c,A,b,Anormed,cnormed,x,index_active)
            index_active = findall(b - A*x .<  atol)
            iter_2 += 1
        end

        (x1,x2) = purification(x,A,b,c,length(x),index_active)
        println("out purification")
        if checkKKT(c,A,b,x1) == 1
            return x1
        else
            x = x2
            index_active = findall(b - A*x .<  atol)
        @show length(index_active)
            d = findDirection(x, A, b, Anormed,cnormed, index_active)
            α = ratiotest(x,A,b,d)
            x = x + α*d
        end
        index_active = findall(b - A*x .<  atol)
        @show iter+=1
#         @show c'*x
    end
    println("Max iterations!")
    return x
end

xo = x_circus
index_active = findall(b - A*xo .<  10^-8)

x = circus_acc(c, A, b, x_circus)

num_active = length(index_active) = 2
num_active = length(index_active) = 3
num_active = length(index_active) = 4
num_active = length(index_active) = 5
num_active = length(index_active) = 6
num_active = length(index_active) = 7
num_active = length(index_active) = 8
num_active = length(index_active) = 9
num_active = length(index_active) = 10
num_active = length(index_active) = 11
num_active = length(index_active) = 12
num_active = length(index_active) = 13
num_active = length(index_active) = 14
num_active = length(index_active) = 15
num_active = length(index_active) = 16
num_active = length(index_active) = 17
num_active = length(index_active) = 18
num_active = length(index_active) = 19
num_active = length(index_active) = 20
num_active = length(index_active) = 21
num_active = length(index_active) = 22
num_active = length(index_active) = 23
num_active = length(index_active) = 24
num_active = length(index_active) = 25
num_active = length(index_active) = 26
num_active = length(index_active)

BoundsError: BoundsError: attempt to access 32×29 Array{Float64,2} at index [Base.Slice(Base.OneTo(32)), 30]

In [62]:
# Saved the case where we get a purification problem
# using CSV, DataFrames
# CSV.write("afiro_A_b_noise.csv",  DataFrame([A b]), writeheader=false)
# CSV.write("afiro_c_x_noise.csv",  DataFrame([c x_circus]), writeheader=false)